# Atividade 01
## Extraia os dados do arquivo 'dados_vendas.csv'. Realize uma análise exploratoria inicial mostrando as:
* dimensoes(linhas e colunas);
* tipos de dados; 
* valores ausentes;
* resumo estatistico;

In [1]:
import pandas as pd
import psycopg2
from datetime import datetime

# Carregar o CSV diretamente do GitHub
url = "https://raw.githubusercontent.com/GabrielSique1r4/Pipeline_de_dados/e34d31a12991a44f58a1aa5210c6c34f51187b52/Aula5/dados_vendas.csv"
df = pd.read_csv(url, encoding='ISO-8859-1', sep=';')

# Análise exploratória
print("Dimensões (linhas, colunas):")
print(df.shape)

print("\nTipos de dados:")
print(df.dtypes)

print("\nValores ausentes por coluna:")
print(df.isnull().sum())

print("\nResumo estatístico:")
print(df.describe(include='all'))


Dimensões (linhas, colunas):
(2015, 38)

Tipos de dados:
id_venda                   int64
id_pedido                  int64
id_produto                 int64
id_cliente                 int64
id_loja                    int64
data_pedido               object
quantidade               float64
preco_unitario           float64
valor_desconto           float64
custo_frete              float64
status_pedido             object
metodo_pagamento          object
nota_fiscal               object
nome_produto              object
categoria_produto         object
subcategoria_produto      object
preco_custo              float64
preco_venda              float64
peso_kg                  float64
fornecedor_id              int64
nome_fornecedor           object
data_cadastro_produto     object
em_estoque                  bool
nome_cliente              object
email_cliente             object
telefone_cliente          object
data_cadastro_cliente     object
segmento_cliente          object
cidade_cliente     

# Atividade 02
## Explore a estrutura dos dados identificando as entidades presentes, suas colunas correspondentes e a cardinalidade das relacoes.
* Identifique as entidades presentes no conjunto de dados;
* Para cada entidade, identifique as colunas correspondentes;
* Verifique a cardinalidade das relacoes analisando valores unicos nas colunas ID

In [2]:
# Entidades e colunas correspondentes (resumido com base no nome das colunas)
print("Entidades identificadas:")
print("- Cliente")
print("- Loja")
print("- Produto")
print("- Pedido")
print("- Venda")

# Exemplo de verificação da cardinalidade
print("\nValores únicos por ID:")
print("Clientes:", df['id_cliente'].nunique())
print("Produtos:", df['id_produto'].nunique())
print("Pedidos:", df['id_pedido'].nunique())
print("Lojas:", df['id_loja'].nunique())


Entidades identificadas:
- Cliente
- Loja
- Produto
- Pedido
- Venda

Valores únicos por ID:
Clientes: 50
Produtos: 100
Pedidos: 391
Lojas: 10


# Atividade 03
## Identifique e corrija os problemas nos dados: valores nulos, quantidades negativas, datas em formato errado, categorias inconsistentes e possivel duplicatas.
* Trate os valores nulos nas colunas 'preco_unitario', 'valor_desconto' e 'nome_fornecedos'  
* Corrija as quantidades negativas.
* Padronize todas as datas para o formato 'YYY-MM-DD'.
* Padronize as categoriaas de produtos (Algumas estao em maiuscula)
* Identifique e trate possiveis duplicatas


In [3]:
# Conversão de datas
df['data_pedido'] = pd.to_datetime(df['data_pedido'], format='%d/%m/%Y', errors='coerce')
df['data_cadastro_produto'] = pd.to_datetime(df['data_cadastro_produto'], format='%d/%m/%Y', errors='coerce')
df['data_cadastro_cliente'] = pd.to_datetime(df['data_cadastro_cliente'], format='%d/%m/%Y', errors='coerce')
df['data_inauguracao'] = pd.to_datetime(df['data_inauguracao'], format='%d/%m/%Y', errors='coerce')

# Tratamento de valores nulos
df.nome_fornecedor = df.nome_fornecedor.fillna('Não identificado')
df.valor_desconto = df.valor_desconto.transform(lambda x: x.fillna(x.mean()))
df.preco_unitario = df.groupby('id_produto')['preco_unitario'].transform(lambda x: x.fillna(x.mean()))
df.quantidade = df.quantidade.transform(lambda x: x.fillna(round(x.mean())))

# Correção de quantidades negativas
df['quantidade'] = df['quantidade'].abs()

# Padronização de categorias
df['categoria_produto'] = df['categoria_produto'].str.capitalize()

# Verificar duplicatas
duplicatas = df.duplicated().sum()
print(f"Número de registros duplicados: {duplicatas}")


Número de registros duplicados: 0


# Atividade 04
## Calcule novas metricas para enriquecer a analise, valor bruto, valor liquido, margem de lucro e dias desde o pedido.
* Valor bruto (quantidade*preco_unitario)
* Valor liquido (valor bruto - valor_desconto)
* Margen de lucro((preco_unitario - preco_custo)/preco_unitario*100)
* Dias desde o pedido (considerando a data atual)

In [4]:
# Campos Calculados
df['valor_bruto'] = df.preco_unitario * df.quantidade
df['valor_liquido'] = df.valor_bruto - df.valor_desconto
df['margem_de_lucro'] = (((df.preco_unitario - df.preco_custo) / df.preco_unitario) * 100)
df['dias_desde_o_pedido'] = (pd.Timestamp.today() - df.data_pedido).dt.days

# Atividade 05
## Identifique e corrija os problemas nos dados: valores nulos, quantidades negativas, datas em formato errado, categorias inconsistentes e possiveis duplicatas.
* Desenhe um diagrama conceitual identificando a tabela fato e as dimensoes.
* Defina as chaves primarias e estrangeiras.
* Determine o nivel de granuladade da tabela fato.
![diagramaConceitual](diagramaConceitual.jpg)
* Idetifique metricas (medidas) e atributos (dimensoes)
### Métricas (Medidas)
* valor_bruto
* valor_liquido
* margem_de_lucro
* quantidade
* valor_desconto
* custo_frete

### Atributos (Dimensões)
* nome_cliente, segmento_cliente, estado_cliente (dim_cliente)
* nome_loja, cidade_loja, tipo_loja (dim_loja)
* nome_produto, categoria_produto, fornecedor_id (dim_produto)
* data_pedido, metodo_pagamento, status_pedido (dim_tempo ou direto na fato)

# Atividade 06
## Implemente a dimencao de produtos (Dim_Produto) selecinando apenas as colunas relacionadas aprodutos, removendo duplicatas e organizando hierarquias.
* Selecione apenas as colunas relacionadas a produtos do dataset.
* Remova duplicatas para garantir que cada produto apareça apenas uma vez.
* Organize hierarquias (categoria > subcategoria).
* Defina a chave primária

In [5]:
# Atividade 06 - Dimensão de Produto

# Seleciona apenas as colunas relacionadas a produtos
dim_produto = df[['id_produto', 'nome_produto', 'categoria_produto', 'subcategoria_produto', 'fornecedor_id']].copy()

# Remove duplicatas para garantir que cada produto apareça apenas uma vez
dim_produto = dim_produto.drop_duplicates()

# Organiza a hierarquia: categoria > subcategoria
dim_produto = dim_produto.sort_values(by=['categoria_produto', 'subcategoria_produto'])

# Define a chave primária logicamente (sem setar como índice)
# id_produto continua como coluna para facilitar merges e validações

# Exibe a dimensão produto
dim_produto.head()


,id_produto,nome_produto,categoria_produto,subcategoria_produto,fornecedor_id
75,4,Produto 4,Alimentos,Bebidas,3
105,77,Produto 77,Alimentos,Bebidas,5
118,20,Produto 20,Alimentos,Bebidas,10
128,60,Produto 60,Alimentos,Bebidas,1
132,35,Produto 35,Alimentos,Bebidas,13


# Atividade 07
## Implemente a dimensão de clientes (Dim_Cliente) selecionando as colunas relacionadas a clientes, removendo duplicatas e organizando hierarquias de localização.
* Selecione apenas as colunas relacionadas a clientes.
* Remova duplicatas.
* Organize hierarquias de localização.
* Defina a chave primária.

In [6]:
# Atividade 07 - Dimensão de Cliente

# Seleciona as colunas relacionadas a clientes
dim_cliente = df[['id_cliente', 'nome_cliente', 'email_cliente', 'telefone_cliente',
                  'data_cadastro_cliente', 'segmento_cliente',
                  'cidade_cliente', 'estado_cliente']].copy()

# Remove duplicatas
dim_cliente = dim_cliente.drop_duplicates()

# Organiza hierarquias de localização: estado > cidade
dim_cliente = dim_cliente.sort_values(by=['estado_cliente', 'cidade_cliente'])

# Define a chave primária
dim_cliente.set_index('id_cliente', inplace=True)

# Exibe os primeiros registros da dimensão cliente
dim_cliente.head()


,nome_cliente,email_cliente,telefone_cliente,data_cadastro_cliente,segmento_cliente,cidade_cliente,estado_cliente
id_cliente,,,,,,,
1,Cliente 1,cliente1@email.com,(11) 92252-7996,2022-03-07,Premium,Belo Horizonte,MG
30,Cliente 30,cliente30@email.com,(11) 96569-5300,2022-05-06,Premium,Belo Horizonte,MG
30,Cliente 30,cliente30@email.com,(11) 92497-3555,2022-10-17,Premium,Belo Horizonte,MG
49,Cliente 49,cliente49@email.com,(11) 95068-4277,2022-04-24,Premium,Belo Horizonte,MG
43,Cliente 43,cliente43@email.com,(11) 95434-1658,2022-09-28,Varejo,Belo Horizonte,MG


# Atividade 08
## Implemente a dimensão de lojas (Dim_Loja) selecionando as colunas relacionadas a lojas, removendo duplicatas e organizando hierarquias geográficas.
* Selecione apenas as colunas relacionadas a lojas.
* Remova duplicatas.
* Organize hierarquias geográficas (região > estado > cidade).
* Defina a chave primária.

In [7]:
# Atividade 08 - Dimensão de Loja

# Seleciona as colunas relacionadas a lojas
dim_loja = df[['id_loja', 'nome_loja', 'endereco_loja', 'cidade_loja',
               'estado_loja', 'regiao_loja', 'gerente_loja',
               'data_inauguracao', 'tipo_loja']].copy()

# Remove duplicatas
dim_loja = dim_loja.drop_duplicates()

# Organiza hierarquias geográficas: região > estado > cidade
dim_loja = dim_loja.sort_values(by=['regiao_loja', 'estado_loja', 'cidade_loja'])

# Define a chave primária
dim_loja.set_index('id_loja', inplace=True)

# Exibe os primeiros registros da dimensão loja
dim_loja.head()


,nome_loja,endereco_loja,cidade_loja,estado_loja,regiao_loja,gerente_loja,data_inauguracao,tipo_loja
id_loja,,,,,,,,
1,Loja 1,Endereço da Loja 1,Belo Horizonte,MG,Nordeste,Gerente 1,2020-06-13,Shopping
6,Loja 6,Endereço da Loja 6,Belo Horizonte,MG,Nordeste,Gerente 6,2018-07-11,Shopping
1,Loja 1,Endereço da Loja 1,Belo Horizonte,MG,Nordeste,Gerente 1,2017-09-15,Rua
4,Loja 4,Endereço da Loja 4,Belo Horizonte,MG,Nordeste,Gerente 4,2016-11-25,Centro Comercial
9,Loja 9,Endereço da Loja 9,Belo Horizonte,MG,Nordeste,Gerente 9,2016-06-04,Centro Comercial


# Atividade 09
## Crie uma dimensão de tempo (Dim_Tempo) a partir das datas de pedido extraindo componentes de data e adicionando indicadores úteis para análises.
* Extraia componentes de data: ano, trimestre, mês, semana, dia, dia da semana.
* Adicione indicadores para fins de semana, feriados (simule alguns).
* Crie classificações por período (manhã, tarde, noite) para análises futuras.
* Considere a criação de campos para análises de sazonalidade.


In [8]:
# Atividade 09 - Dimensão de Tempo

# Garante que a coluna de data esteja no formato datetime
df['data_pedido'] = pd.to_datetime(df['data_pedido'])

# Remove duplicatas de datas
dim_tempo = pd.DataFrame(df['data_pedido'].drop_duplicates()).copy()

# Criação dos componentes de data
dim_tempo['ano'] = dim_tempo['data_pedido'].dt.year
dim_tempo['trimestre'] = dim_tempo['data_pedido'].dt.quarter
dim_tempo['mes'] = dim_tempo['data_pedido'].dt.month
dim_tempo['nome_mes'] = dim_tempo['data_pedido'].dt.month_name()
dim_tempo['semana'] = dim_tempo['data_pedido'].dt.isocalendar().week
dim_tempo['dia'] = dim_tempo['data_pedido'].dt.day
dim_tempo['dia_semana'] = dim_tempo['data_pedido'].dt.day_name()

# Indicador de fim de semana
dim_tempo['fim_de_semana'] = dim_tempo['dia_semana'].isin(['Saturday', 'Sunday'])

# Simula feriados (exemplo: 1º de janeiro, 7 de setembro, 25 de dezembro)
feriados_simulados = [
    '2023-01-01', '2023-09-07', '2023-12-25',
    '2024-01-01', '2024-09-07', '2024-12-25'
]
feriados_simulados = pd.to_datetime(feriados_simulados)
dim_tempo['feriado'] = dim_tempo['data_pedido'].isin(feriados_simulados)

# Classificação por período do dia (baseado na hora)
# Se a coluna de hora não existir, vamos simular um horário aleatório
if not hasattr(dim_tempo['data_pedido'].dt, 'hour'):
    # Adiciona hora aleatória entre 0 e 23 se não existir
    dim_tempo['hora_simulada'] = np.random.randint(0, 24, dim_tempo.shape[0])
else:
    dim_tempo['hora_simulada'] = dim_tempo['data_pedido'].dt.hour

def classificar_periodo(hora):
    if hora < 12:
        return 'Manhã'
    elif hora < 18:
        return 'Tarde'
    else:
        return 'Noite'

dim_tempo['periodo_dia'] = dim_tempo['hora_simulada'].apply(classificar_periodo)

# Campo para sazonalidade: Exemplo, alta/baixa temporada
dim_tempo['sazonalidade'] = dim_tempo['mes'].apply(lambda x: 'Alta' if x in [11, 12, 1, 7] else 'Baixa')

# Define a chave primária
dim_tempo.set_index('data_pedido', inplace=True)

# Exibe os primeiros registros da dimensão tempo
dim_tempo.head()


,ano,trimestre,mes,nome_mes,semana,dia,dia_semana,fim_de_semana,feriado,hora_simulada,periodo_dia,sazonalidade
data_pedido,,,,,,,,,,,,
2023-12-10,2023.0,4.0,12.0,December,49,10.0,Sunday,True,False,0.0,Manhã,Alta
2023-09-21,2023.0,3.0,9.0,September,38,21.0,Thursday,False,False,0.0,Manhã,Baixa
2023-05-16,2023.0,2.0,5.0,May,20,16.0,Tuesday,False,False,0.0,Manhã,Baixa
2023-12-25,2023.0,4.0,12.0,December,52,25.0,Monday,False,True,0.0,Manhã,Alta
2023-05-08,2023.0,2.0,5.0,May,19,8.0,Monday,False,False,0.0,Manhã,Baixa


# Atividade 10
## Implemente a tabela fato de vendas (Fato_Vendas) selecionando as métricas numéricas, incluindo as chaves estrangeiras para todas as dimensões e mantendo apenas os dados necessários.
* Selecione as métricas numéricas (quantidade, valores, etc.).
* Inclua as chaves estrangeiras para todas as dimensões.
* Mantenha apenas os dados necessários no nível de granularidade definido.
* Valide a integridade referencial com as dimensões criadas.

In [9]:
# Seleciona as colunas que compõem a tabela fato
fato_vendas = df[[
    'id_cliente',
    'id_pedido',
    'id_produto',
    'id_loja',
    'quantidade',
    'preco_unitario',
    'valor_bruto',
    'valor_desconto',
    'valor_liquido',
    'custo_frete',
    'margem_de_lucro',
    'dias_desde_o_pedido'
]].copy()

# Remove duplicatas, se houver
fato_vendas = fato_vendas.drop_duplicates()

# Exibe o início do DataFrame para conferência
fato_vendas.head()


,id_cliente,id_pedido,id_produto,id_loja,quantidade,preco_unitario,valor_bruto,valor_desconto,valor_liquido,custo_frete,margem_de_lucro,dias_desde_o_pedido
0,19,1,52,8,6.0,540.89,3245.34,973.60,2271.74,32.79,27.720978,487.0
1,50,1,21,9,3.0,1045.06,3135.18,250.81,2884.37,34.81,54.576771,567.0
2,41,1,53,1,8.0,494.08,3952.64,237.16,3715.48,37.01,70.842778,695.0
3,14,1,86,1,9.0,133.81,1204.29,36.13,1168.16,24.77,-78.424632,472.0
4,45,1,71,9,7.0,1004.10,7028.70,702.87,6325.83,9.95,91.554626,703.0


# Atividade 11
## Construa o esquema no SQLite para o data warehouse, definindo tabelas, chaves primárias, chaves estrangeiras e índices para melhorar performance.
* Crie o banco de dados 'dw_vendas.sqlite'.
* Defina o esquema para todas as tabelas dimensionais e de fatos.
* Estabeleça chaves primárias e estrangeiras.

In [10]:
import sqlite3

# Conectar (ou criar) o banco de dados SQLite
conn = sqlite3.connect('dw_vendas.sqlite')
cursor = conn.cursor()

# Criação das tabelas dimensionais e fato com chaves e índices
cursor.executescript("""
-- Tabela Cliente
CREATE TABLE IF NOT EXISTS cliente (
    id_cliente INTEGER PRIMARY KEY,
    nome_cliente TEXT,
    email_cliente TEXT,
    telefone_cliente TEXT,
    data_cadastro_cliente DATE,
    segmento_cliente TEXT,
    cidade_cliente TEXT,
    estado_cliente TEXT
);

-- Tabela Loja
CREATE TABLE IF NOT EXISTS loja (
    id_loja INTEGER PRIMARY KEY,
    nome_loja TEXT,
    endereco_loja TEXT,
    cidade_loja TEXT,
    estado_loja TEXT,
    regiao_loja TEXT,
    gerente_loja TEXT,
    data_inauguracao_loja DATE,
    tipo_loja TEXT
);

-- Tabela Produto
CREATE TABLE IF NOT EXISTS produto (
    id_produto INTEGER PRIMARY KEY,
    nome_produto TEXT,
    categoria_produto TEXT,
    subcategoria_produto TEXT,
    preco_custo REAL,
    preco_venda REAL,
    peso_kg REAL,
    fornecedor_id INTEGER,
    nome_fornecedor TEXT,
    data_cadastro_produto DATE,
    em_estoque BOOLEAN,
    margem_lucro REAL
);

-- Tabela Pedido
CREATE TABLE IF NOT EXISTS pedido (
    id_pedido INTEGER PRIMARY KEY,
    data_pedido DATE,
    quantidade REAL,
    preco_unitario REAL,
    valor_desconto REAL,
    custo_frete REAL,
    status_pedido TEXT,
    metodo_pagamento TEXT,
    nota_fiscal TEXT,
    dias_desde_pedido INTEGER,
    valor_bruto REAL,
    valor_liquido REAL
);

-- Tabela Produto_Pedido (Relacionamento N-N)
CREATE TABLE IF NOT EXISTS produto_pedido (
    id_produto_pedido INTEGER PRIMARY KEY AUTOINCREMENT,
    id_pedido INTEGER,
    id_produto INTEGER,
    FOREIGN KEY (id_pedido) REFERENCES pedido(id_pedido),
    FOREIGN KEY (id_produto) REFERENCES produto(id_produto)
);

-- Tabela Fato Venda
CREATE TABLE IF NOT EXISTS venda (
    id_venda INTEGER PRIMARY KEY,
    id_cliente INTEGER,
    id_pedido INTEGER,
    id_loja INTEGER,
    FOREIGN KEY (id_cliente) REFERENCES cliente(id_cliente),
    FOREIGN KEY (id_pedido) REFERENCES pedido(id_pedido),
    FOREIGN KEY (id_loja) REFERENCES loja(id_loja)
);

-- Índices para performance
CREATE INDEX IF NOT EXISTS idx_venda_cliente ON venda(id_cliente);
CREATE INDEX IF NOT EXISTS idx_venda_loja ON venda(id_loja);
CREATE INDEX IF NOT EXISTS idx_venda_pedido ON venda(id_pedido);
""")

# Commit e fechar conexão
conn.commit()
conn.close()


# Atividade 12
## Carregue os dados processados no banco de dados SQLite, primeiro as dimensões e depois as tabelas fato, validando a integridade dos dados carregados.
* Carregue todas as dimensões primeiro.
* Carregue a tabela fato principal.

In [11]:
import sqlite3

# Conectar ao SQLite
conn = sqlite3.connect('dw_vendas.sqlite')
cursor = conn.cursor()

# --- Carregar DataFrames corrigidos ---

# Cliente
df_cliente = df[[  
    'id_cliente', 'nome_cliente', 'email_cliente', 'telefone_cliente',
    'data_cadastro_cliente', 'segmento_cliente', 'cidade_cliente', 'estado_cliente'
]].drop_duplicates()

# Loja
df_loja = df[[  
    'id_loja', 'nome_loja', 'endereco_loja', 'cidade_loja',
    'estado_loja', 'regiao_loja', 'gerente_loja', 'data_inauguracao', 'tipo_loja'
]].drop_duplicates().rename(columns={
    'data_inauguracao': 'data_inauguracao_loja'
})

# Produto
df_produto = df[[  
    'id_produto', 'nome_produto', 'categoria_produto', 'subcategoria_produto',
    'preco_custo', 'preco_unitario', 'peso_kg', 'fornecedor_id', 'nome_fornecedor',
    'data_cadastro_produto', 'em_estoque', 'margem_de_lucro'
]].drop_duplicates().rename(columns={
    'preco_unitario': 'preco_venda',
    'margem_de_lucro': 'margem_lucro'  # Corrigindo nome da coluna para o nome da tabela
})

# Pedido
df_pedido = df[[  
    'id_pedido', 'data_pedido', 'quantidade', 'preco_unitario', 'valor_desconto',
    'custo_frete', 'status_pedido', 'metodo_pagamento', 'nota_fiscal',
    'dias_desde_o_pedido', 'valor_bruto', 'valor_liquido'
]].drop_duplicates().rename(columns={
    'dias_desde_o_pedido': 'dias_desde_pedido'
})

# Fato Venda
df_venda = df[[  
    'id_venda', 'id_cliente', 'id_pedido', 'id_loja'
]].drop_duplicates(subset='id_venda')

# Relacionamento produto - pedido
df_produto_pedido = df[[  
    'id_pedido', 'id_produto'
]].drop_duplicates()

# --- Função para inserir evitando duplicatas ---

def inserir_sem_duplicar(df, nome_tabela, conn):
    temp_table = f'temp_{nome_tabela}'
    df.to_sql(temp_table, conn, if_exists='replace', index=False)

    colunas = ', '.join(df.columns)
    cursor = conn.cursor()
    cursor.execute(f"""
        INSERT OR IGNORE INTO {nome_tabela} ({colunas})
        SELECT {colunas} FROM {temp_table}
    """)
    conn.commit()
    cursor.execute(f"DROP TABLE {temp_table}")
    conn.commit()

# --- Inserir dados nas tabelas ---

inserir_sem_duplicar(df_cliente, 'cliente', conn)
inserir_sem_duplicar(df_loja, 'loja', conn)
inserir_sem_duplicar(df_produto, 'produto', conn)
inserir_sem_duplicar(df_pedido, 'pedido', conn)
inserir_sem_duplicar(df_venda, 'venda', conn)
inserir_sem_duplicar(df_produto_pedido, 'produto_pedido', conn)

# --- Verificação de integridade ---
cursor.execute("""
    SELECT COUNT(*) FROM venda v
    LEFT JOIN cliente c ON v.id_cliente = c.id_cliente
    WHERE c.id_cliente IS NULL
""")
missing_clientes = cursor.fetchone()[0]
print(f"⚠️ Vendas com cliente inexistente: {missing_clientes}")

# Fechar conexão
conn.close()


⚠️ Vendas com cliente inexistente: 0


# Bonus Integracao com postgres

In [ ]:
import pandas as pd
import psycopg2
from datetime import datetime

# Carregar o CSV diretamente do GitHub.
url = "https://raw.githubusercontent.com/GabrielSique1r4/Pipeline_de_dados/e34d31a12991a44f58a1aa5210c6c34f51187b52/Aula5/dados_vendas.csv"
df = pd.read_csv(url, encoding='ISO-8859-1', sep=';')

# Pré-processamento
df['data_pedido'] = pd.to_datetime(df['data_pedido'], format='%d/%m/%Y', errors='coerce')
df['data_cadastro_produto'] = pd.to_datetime(df['data_cadastro_produto'], format='%d/%m/%Y', errors='coerce')
df['data_cadastro_cliente'] = pd.to_datetime(df['data_cadastro_cliente'], format='%d/%m/%Y', errors='coerce')
df['data_inauguracao'] = pd.to_datetime(df['data_inauguracao'], format='%d/%m/%Y', errors='coerce')

# Tratamento dos Dados
df.nome_fornecedor = df.nome_fornecedor.fillna('Não identificado')
df.valor_desconto = df.valor_desconto.transform(lambda x: x.fillna(x.mean()))
df.preco_unitario = df.groupby('id_produto')['preco_unitario'].transform(lambda x: x.fillna(x.mean()))
df.quantidade = df.quantidade.transform(lambda x: x.fillna(round(x.mean())))
df['quantidade'] = df['quantidade'].abs()
df['categoria_produto'] = df['categoria_produto'].str.capitalize()

# Campos Calculados
df['valor_bruto'] = df.preco_unitario * df.quantidade
df['valor_liquido'] = df.valor_bruto - df.valor_desconto
df['margem_de_lucro'] = (((df.preco_unitario - df.preco_custo) / df.preco_unitario) * 100)
df['dias_desde_o_pedido'] = (pd.Timestamp.today() - df.data_pedido).dt.days

# Conexão ao PostgreSQL
conn = psycopg2.connect(
    host="dpg-cvo8bn3uibrs73blqqpg-a.oregon-postgres.render.com",
    user="dados_venda_user",
    password="WnWOMu47eGyYl5nzacnnEdCjyJACS3TK",
    database="dados_venda"
)
conn.autocommit = True
cursor = conn.cursor()

# Criação das tabelas
cursor.execute("""
CREATE TABLE IF NOT EXISTS cliente(
    id_cliente int PRIMARY KEY,
    nome_cliente varchar(50),
    email_cliente varchar(150),
    telefone_cliente varchar(20),
    data_cadastro_cliente date,
    segmento_cliente varchar(25),
    cidade_cliente varchar(80),
    estado_cliente varchar(25)
);

CREATE TABLE IF NOT EXISTS loja(
    id_loja int PRIMARY KEY,
    nome_loja varchar(100),
    endereco_loja varchar(250),
    cidade_loja varchar(80),
    estado_loja varchar(25),
    regiao_loja varchar(15),
    gerente_loja varchar(50),
    data_inauguracao_loja date,
    tipo_loja varchar(30)
);

CREATE TABLE IF NOT EXISTS produto(
    id_produto int PRIMARY KEY,
    nome_produto varchar(100),
    categoria_produto varchar(50),
    subcategoria_produto varchar(50),
    preco_custo decimal(7,2),
    preco_venda decimal(7,2),
    peso_kg decimal(6,2),
    fornecedor_id int,
    nome_fornecedor varchar(60),
    data_cadastro_produto date,
    em_estoque boolean,
    margem_lucro decimal(6,2)
);

CREATE TABLE IF NOT EXISTS pedido(
    id_pedido int PRIMARY KEY,
    data_pedido date,
    quantidade decimal(4,1),
    preco_unitario decimal(7,2),
    valor_desconto decimal(7,2),
    custo_frete decimal(7,2),
    status_pedido varchar(25),
    metodo_pagamento varchar(30),
    nota_fiscal varchar(20),
    dias_desde_pedido int,
    valor_bruto decimal(8,2),
    valor_liquido decimal(8,2)
);

CREATE TABLE IF NOT EXISTS produto_pedido(
    id_produto_pedido serial PRIMARY KEY,
    id_pedido INT REFERENCES pedido(id_pedido),
    id_produto INT REFERENCES produto(id_produto)
);

CREATE TABLE IF NOT EXISTS venda(
    id_venda int PRIMARY KEY,
    id_cliente int REFERENCES cliente(id_cliente),
    id_pedido int REFERENCES pedido(id_pedido),
    id_loja int REFERENCES loja(id_loja)
);
""")

# Fechar conexão
cursor.close()
conn.close()

In [ ]:
import sqlite3

# Conectar ao SQLite
conn = sqlite3.connect('dw_vendas.sqlite')
cursor = conn.cursor()

# --- Carregar DataFrames corrigidos ---

# Cliente
df_cliente = df[[  
    'id_cliente', 'nome_cliente', 'email_cliente', 'telefone_cliente',
    'data_cadastro_cliente', 'segmento_cliente', 'cidade_cliente', 'estado_cliente'
]].drop_duplicates()

# Loja
df_loja = df[[  
    'id_loja', 'nome_loja', 'endereco_loja', 'cidade_loja',
    'estado_loja', 'regiao_loja', 'gerente_loja', 'data_inauguracao', 'tipo_loja'
]].drop_duplicates().rename(columns={
    'data_inauguracao': 'data_inauguracao_loja'
})

# Produto
df_produto = df[[  
    'id_produto', 'nome_produto', 'categoria_produto', 'subcategoria_produto',
    'preco_custo', 'preco_unitario', 'peso_kg', 'fornecedor_id', 'nome_fornecedor',
    'data_cadastro_produto', 'em_estoque', 'margem_de_lucro'
]].drop_duplicates().rename(columns={
    'preco_unitario': 'preco_venda',
    'margem_de_lucro': 'margem_lucro'  # Corrigindo nome da coluna para o nome da tabela
})

# Pedido
df_pedido = df[[  
    'id_pedido', 'data_pedido', 'quantidade', 'preco_unitario', 'valor_desconto',
    'custo_frete', 'status_pedido', 'metodo_pagamento', 'nota_fiscal',
    'dias_desde_o_pedido', 'valor_bruto', 'valor_liquido'
]].drop_duplicates().rename(columns={
    'dias_desde_o_pedido': 'dias_desde_pedido'
})

# Fato Venda
df_venda = df[[  
    'id_venda', 'id_cliente', 'id_pedido', 'id_loja'
]].drop_duplicates(subset='id_venda')

# Relacionamento produto - pedido
df_produto_pedido = df[[  
    'id_pedido', 'id_produto'
]].drop_duplicates()

# --- Função para inserir evitando duplicatas ---

def inserir_sem_duplicar(df, nome_tabela, conn):
    temp_table = f'temp_{nome_tabela}'
    df.to_sql(temp_table, conn, if_exists='replace', index=False)

    colunas = ', '.join(df.columns)
    cursor = conn.cursor()
    cursor.execute(f"""
        INSERT OR IGNORE INTO {nome_tabela} ({colunas})
        SELECT {colunas} FROM {temp_table}
    """)
    conn.commit()
    cursor.execute(f"DROP TABLE {temp_table}")
    conn.commit()

# --- Inserir dados nas tabelas ---

inserir_sem_duplicar(df_cliente, 'cliente', conn)
inserir_sem_duplicar(df_loja, 'loja', conn)
inserir_sem_duplicar(df_produto, 'produto', conn)
inserir_sem_duplicar(df_pedido, 'pedido', conn)
inserir_sem_duplicar(df_venda, 'venda', conn)
inserir_sem_duplicar(df_produto_pedido, 'produto_pedido', conn)

# --- Verificação de integridade ---
cursor.execute("""
    SELECT COUNT(*) FROM venda v
    LEFT JOIN cliente c ON v.id_cliente = c.id_cliente
    WHERE c.id_cliente IS NULL
""")
missing_clientes = cursor.fetchone()[0]
print(f"⚠️ Vendas com cliente inexistente: {missing_clientes}")

# Fechar conexão
conn.close()


⚠️ Vendas com cliente inexistente: 0
